In [1]:
%matplotlib inline
import matplotlib as mpl
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
#from sklearn.preprocessing import Imputer ==> classe excluida do sklearn, sendo substituida pela SimpleImputer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from functools import partial
from modAL.batch import uncertainty_batch_sampling


In [2]:
#carrega o dataset
dados = pd.read_csv('/home/erasmor/Área de Trabalho/CSE-CIC-IDS2018/500amostras_DDoS.csv',sep=",",encoding = 'utf-8',  header=0,na_values='.')
#remove valores infinitos
dados.replace(-np.Inf, np.nan)
#substitui valores NaN
dados.fillna(dados.mean())

,Dst Port,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,3389,6,3360847,10,7,1132,1581,661,0,113.2,...,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign
1,53,17,3028,1,1,49,164,49,49,49.0,...,8,0.0,0.0,0,0,0.0,0.0,0,0,Benign
2,51019,6,129,2,0,0,0,0,0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign
3,54395,6,33,2,0,0,0,0,0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign
4,38351,6,86403715,2,0,0,0,0,0,0.0,...,20,0.0,0.0,0,0,86403715.0,0.0,86403715,86403715,Benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,80,17,91582627,247725,0,7927200,0,32,32,32.0,...,8,0.0,0.0,0,0,0.0,0.0,0,0,DDOS attack-LOIC-UDP
1496,80,17,81660512,93455,0,2990560,0,32,32,32.0,...,8,0.0,0.0,0,0,0.0,0.0,0,0,DDOS attack-LOIC-UDP
1497,80,17,92865631,115936,0,3709952,0,32,32,32.0,...,8,0.0,0.0,0,0,0.0,0.0,0,0,DDOS attack-LOIC-UDP
1498,80,17,77399404,68375,0,2188000,0,32,32,32.0,...,8,0.0,0.0,0,0,0.0,0.0,0,0,DDOS attack-LOIC-UDP


In [3]:
#mostra como está a base de dados
dados.head()

,Dst Port,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,3389,6,3360847,10,7,1132,1581,661,0,113.2,...,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign
1,53,17,3028,1,1,49,164,49,49,49.0,...,8,0.0,0.0,0,0,0.0,0.0,0,0,Benign
2,51019,6,129,2,0,0,0,0,0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign
3,54395,6,33,2,0,0,0,0,0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign
4,38351,6,86403715,2,0,0,0,0,0,0.0,...,20,0.0,0.0,0,0,86403715.0,0.0,86403715,86403715,Benign


In [4]:
# verifica quantas instâncias (linhas) e quantos atributos (colunas) a base de dados contém
print("numero de linhas e colunas: ",dados.shape)

numero de linhas e colunas:  (1500, 79)


In [5]:
#visualizar distribições por classes contidas no csv - informar nome da classe alvo
print(dados.groupby('Label').size())

Label
Benign                  500
DDOS attack-HOIC        500
DDOS attack-LOIC-UDP    500
dtype: int64


In [6]:
#define as colunas de atributos e a coluna da classe (de 0 a 72 são atributos e após a 72 é a classe)
# "X_raw" é features/atributos e "y_raw" é target/classe ==> As duas formas abaixo dão certo.
#array = dataset.values
#X_raw = array[:,0:72]
#y_raw = array[:,72]
X_raw = dados.iloc[:, :-1].values # atributos
y_raw = dados.iloc[:, 78].values # classe de ataques


In [7]:
X_raw

array([[3.3890000e+03, 6.0000000e+00, 3.3608470e+06, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [5.3000000e+01, 1.7000000e+01, 3.0280000e+03, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [5.1019000e+04, 6.0000000e+00, 1.2900000e+02, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       ...,
       [8.0000000e+01, 1.7000000e+01, 9.2865631e+07, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [8.0000000e+01, 1.7000000e+01, 7.7399404e+07, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [8.0000000e+01, 1.7000000e+01, 7.5930004e+07, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00]])

In [8]:
y_raw

array(['Benign', 'Benign', 'Benign', ..., 'DDOS attack-LOIC-UDP',
       'DDOS attack-LOIC-UDP', 'DDOS attack-LOIC-UDP'], dtype=object)

In [9]:
#transformar a variável Y com valores categóricos das classses de ataques em valores:
labelencoder_y = LabelEncoder()
y_raw = labelencoder_y.fit_transform(y_raw)

In [10]:
y_raw

array([0, 0, 0, ..., 2, 2, 2])

In [11]:
#X_raw_normalize = MinMaxScaler(X_raw_normalize.reshape(0, 1)).reshape(len(X_raw_normalize))
#X_raw_normalizetd2 = (X_raw_normalize - X_raw_normalize.min(axis=0)) / (X_raw_normalize.max(axis=0) - X_raw_normalize.min(axis=0))
# Obtendo os nomes das colunas do DataFrame como uma lista.
cols = list(dados.columns)
# colunas que nao serao normalizadas
cols.remove('Label')
# Copiando os dados e aplicando a normalizacao por reescala nas colunas do DataFrame que contem
# valores continuos. Por padrao, o metodo minmax_scale reescala com min=0 e max=1.
dados = dados[~dados.isin([np.nan, np.inf, -np.inf]).any(1)]
dados[cols] = dados[cols].apply(minmax_scale)
#dados_amp = dados.copy()
#dados_amp[cols] = dados[cols].apply(minmax_scale)
#print (dados_dist.dtypes)
#dados_amp[cols]
dados[cols]

,Dst Port,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,...,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,0.052220,0.352941,2.800705e-02,0.000036,0.500000,0.000143,0.336670,0.594424,0.000000,0.516895,...,0.000020,0.5,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000
1,0.000817,1.000000,2.522500e-05,0.000000,0.071429,0.000006,0.034923,0.044065,0.583333,0.223744,...,0.000000,0.2,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000
2,0.786141,0.352941,1.066667e-06,0.000004,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.5,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000
3,0.838161,0.352941,2.666667e-07,0.000004,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.5,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000
4,0.590943,0.352941,7.200310e-01,0.000004,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.5,0.0,0.0,0.0,0.0,0.995073,0.0,0.995073,0.995073
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,0.001233,1.000000,7.631886e-01,1.000000,0.000000,1.000000,0.000000,0.028777,0.380952,0.146119,...,1.000000,0.2,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000
1496,0.001233,1.000000,6.805043e-01,0.377250,0.000000,0.377253,0.000000,0.028777,0.380952,0.146119,...,0.377250,0.2,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000
1497,0.001233,1.000000,7.738803e-01,0.468001,0.000000,0.468003,0.000000,0.028777,0.380952,0.146119,...,0.468001,0.2,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000
1498,0.001233,1.000000,6.449951e-01,0.276009,0.000000,0.276012,0.000000,0.028777,0.380952,0.146119,...,0.276009,0.2,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000


In [12]:
# Instanciando um PCA. O parametro n_components indica a quantidade de dimensoes que a base
# original sera reduzida.
pca = PCA(n_components=2, whiten=True)

In [13]:
# Aplicando o pca na base de dados. O atributo 'values' retorna um numpy.array
# de duas dimensões (matriz) contendo apenas os valores numericos do DataFrame.
dados_pca = pca.fit_transform(dados[cols].values)

In [14]:
dados_pca

array([[-0.52821002, -2.32840444],
       [ 0.09272212, -0.87777276],
       [-0.85285485,  0.01409218],
       ...,
       [ 1.25159179,  0.08918905],
       [ 0.89453007, -0.07164474],
       [ 0.8954248 , -0.07796954]])

In [ ]:
# Isolate the data we'll need for plotting.
x_component, y_component = dados_pca[:, 0], dados_pca[:, 1]
colormap = {'Benign': 'red', 'DDOS attack-HOIC': 'green', 'DDOS attack-LOIC-UDP': 'blue'}
colors = [colormap[x] for x in dados['Label']]
# Plot our dimensionality-reduced (via PCA) dataset.
plt.figure(figsize=(8.5, 6), dpi=130)
plt.scatter(x=x_component, y=y_component, c = colors, cmap='viridis', s=10, alpha=8/10)
plt.title('Classes de Ataques após transformação PCA')
plt.show()

In [ ]:
#Particionamento do conjunto de dados em um conjunto de treinamento L e U. Primeiro especificamos
#conjunto de treinamento L consistindo em 3 exemplos aleatórios. Os exemplos restantes vão para
#o pool "sem rótulo" U.

#Isole o exemplos para o conjunto de dados rotulado:
n_labeled_examples = X_raw.shape[0]
training_indices = np.random.randint(low=0, high=n_labeled_examples + 1, size=3)

X_train = X_raw[training_indices]
y_train = y_raw[training_indices]

#Isola os exemplos sem treinamento para consultar.
X_pool = np.delete(X_raw, training_indices, axis=0)
y_pool = np.delete(y_raw, training_indices, axis=0)
#teste
dados.info()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from modAL.models import ActiveLearner
from functools import partial
from modAL.batch import uncertainty_batch_sampling
from sklearn.impute import SimpleImputer


# Especifique o estimador principal (K-NN) junto com o modelo de aprendizado ativo.
knn = KNeighborsClassifier(n_neighbors=3)
# Predefina a amostragem de lote para recuperar 3 amostras por vez.
BATCH_SIZE = 3
preset_batch = partial(uncertainty_batch_sampling, n_instances=BATCH_SIZE)

# Especifique o modelo de aprendizagem ativa.
learner = ActiveLearner(estimator=knn,X_training=X_train,y_training=y_train,query_strategy=preset_batch)


In [ ]:
# Isole os dados para plotar.
predictions = learner.predict(X_raw)
is_correct = (predictions == y_raw)

# Registre a pontuação do learner nos dados brutos.
unqueried_score = learner.score(X_raw, y_raw)

In [ ]:
# Trace os resultados de classificação.
with plt.style.context('seaborn-white'):
    fig, ax = plt.subplots(figsize=(8.5, 6), dpi=130)
    ax.scatter(x=x_component[is_correct],  y=y_component[is_correct],  c='g', marker='+', label='Correct')
    ax.scatter(x=x_component[~is_correct], y=y_component[~is_correct], c='r', marker='x', label='Incorrect')
    ax.legend(loc='lower right')
    ax.set_title("ActiveLearner predições de classe (Accuracy: {score:.3f})".format(score=unqueried_score))
    plt.show()

In [ ]:
# Atualize o modelo por meio de amostragem baseada em pool do conjunto de dados "sem etiqueta" U
# Pool-based sampling
N_RAW_SAMPLES = 20
N_QUERIES = N_RAW_SAMPLES // BATCH_SIZE
performance_history = [unqueried_score]

for index in range(N_QUERIES):
    query_index, query_instance = learner.query(X_pool)

    # Teach our ActiveLearner model the record it has requested.
    X, y = X_pool[query_index], y_pool[query_index]
    learner.teach(X=X, y=y)

    # Remove the queried instance from the unlabeled pool.
    X_pool = np.delete(X_pool, query_index, axis=0)
    y_pool = np.delete(y_pool, query_index)

    # Calculate and report our model's accuracy.
    model_accuracy = learner.score(X_raw, y_raw)
    print('Accuracy after query {n}: {acc:0.4f}'.format(n=index + 1, acc=model_accuracy))

    # Save our model's performance for plotting.
    performance_history.append(model_accuracy)

    

In [ ]:
# Plot our performance over time.
with plt.style.context('seaborn-white'):
    fig, ax = plt.subplots(figsize=(8.5, 6), dpi=130)

    ax.plot(performance_history)
    ax.scatter(range(len(performance_history)), performance_history, s=13)

    ax.xaxis.set_major_locator(mpl.ticker.MaxNLocator(nbins=N_QUERIES + 3, integer=True))
    ax.xaxis.grid(True)

    ax.yaxis.set_major_locator(mpl.ticker.MaxNLocator(nbins=10))
    ax.yaxis.set_major_formatter(mpl.ticker.PercentFormatter(xmax=1))
    ax.set_ylim(bottom=0, top=1)
    ax.yaxis.grid(True, linestyle='--', alpha=1/2)

    ax.set_title('Incremental classification accuracy')
    ax.set_xlabel('Query iteration')
    ax.set_ylabel('Classification Accuracy')

    plt.show()

In [ ]:
# Isolate the data we'll need for plotting.
predictions = learner.predict(X_raw)
is_correct = (predictions == y_raw)

# Plot our updated classification results once we've trained our learner.
with plt.style.context('seaborn-white'):
    fig, ax = plt.subplots(figsize=(8.5, 6), dpi=130)

    ax.scatter(x=x_component[is_correct],  y=y_component[is_correct],  c='g', marker='+', label='Correct')
    ax.scatter(x=x_component[~is_correct], y=y_component[~is_correct], c='r', marker='x', label='Incorrect')

    ax.set_title('Classification accuracy after {n} queries (* {batch_size} samples/query = {total} samples): {final_acc:.3f}'.format(
        n=N_QUERIES,
        batch_size=BATCH_SIZE,
        total=N_QUERIES * BATCH_SIZE,
        final_acc=performance_history[-1]
    ))
    ax.legend(loc='lower right')

    plt.show()